In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Grafikleri daha temiz göstermek için stil ayarı
sns.set_theme(style="whitegrid")

In [ ]:
plt.figure(figsize=(10, 6))
# Yenilgi sayısı ile oyun süresi arasındaki ilişkiyi gösteren bir scatter plot
sns.regplot(data=df, x='defeat_count', y='total_session_duration', scatter_kws={'alpha':0.3})
plt.title('Yenilgi Sayısının Toplam Oturum Süresine Etkisi')
plt.xlabel('Yenilgi Sayısı')
plt.ylabel('Toplam Oturum Süresi (Saniye)')
plt.show()

In [ ]:
# Gelir türlerini özetleyelim
total_iap = df['iap_revenue'].sum()
total_ad = df['ad_revenue'].sum()

labels = ['IAP (Satın Alma)', 'Ad (Reklam)']
sizes = [total_iap, total_ad]

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff'])
plt.title('Toplam Gelir Dağılımı')
plt.show()

In [ ]:
# Günlük toplam geliri hesapla
daily_revenue = df.groupby('event_date')[['iap_revenue', 'ad_revenue']].sum().reset_index()

plt.figure(figsize=(12, 6))
plt.plot(daily_revenue['event_date'], daily_revenue['iap_revenue'], label='Satın Alma Geliri', marker='o')
plt.plot(daily_revenue['event_date'], daily_revenue['ad_revenue'], label='Reklam Geliri', marker='s')
plt.xticks(rotation=45)
plt.title('Günlük Gelir Trendi')
plt.legend()
plt.show()

In [ ]:
# Ülke bazında ARPU (Kullanıcı Başına Ortalama Gelir)
top_countries = df.groupby('country').agg({
    'iap_revenue': 'sum',
    'user_id': 'nunique'
}).reset_index()

top_countries['ARPU'] = top_countries['iap_revenue'] / top_countries['user_id']
print(top_countries.sort_values('ARPU', ascending=False).head(10))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Sadece mağlubiyet yaşayan (victory_count == 0) satırları alalım
#  match_start_count > 0 olmalı ki maça girmiş olsun
losers_df = df[(df['victory_count'] == 0) & (df['match_start_count'] > 0)].copy()

# 2. Bu grup içindeki "En Aktif" oyuncuları tanımlayalım.
# Session duration ve match_start_count değerleri ortalamanın (median) üzerinde olanlar
# Bunlar "hakkı yenen" potansiyel SVP adaylarıdır.
duration_threshold = losers_df['total_session_duration'].median()
match_threshold = losers_df['match_start_count'].median()

# SVP Adayları: Kaybetmesine rağmen maç sayısı ve süresi yüksek olanlar
svp_candidates = losers_df[
    (losers_df['total_session_duration'] > duration_threshold) & 
    (losers_df['match_start_count'] > match_threshold)
]

# 3. İstatistiksel 
print(f"Toplam Kaybeden Kaydı: {len(losers_df)}")
print(f"Hakkı Yenen (SVP Adayı) Sayısı: {len(svp_candidates)}")
print(f"Kaybedenler içinde 'High-Effort' oyuncu oranı: %{len(svp_candidates)/len(losers_df)*100:.2f}")

# 4. Görselleştirme: Kaybedenler İçindeki "Emek" Dağılımı
plt.figure(figsize=(10, 6))
sns.scatterplot(data=losers_df.sample(2000), x='match_start_count', y='total_session_duration', alpha=0.5)
plt.axvline(match_threshold, color='red', linestyle='--', label='Maç Eşiği (Median)')
plt.axhline(duration_threshold, color='green', linestyle='--', label='Süre Eşiği (Median)')
plt.title('Kaybeden Oyuncuların Çaba Analizi (SVP Potansiyeli)')
plt.xlabel('Maç Başlangıç Sayısı')
plt.ylabel('Toplam Oturum Süresi (Saniye)')
plt.legend()
plt.show()

In [ ]:
# 1. Kazanma Oranı (Win Rate) hesaplayalım
# Sıfıra bölünme hatasını önlemek için küçük bir kontrol ekliyoruz
df['total_games'] = df['victory_count'] + df['defeat_count']
df_active = df[df['total_games'] > 0].copy()
df_active['win_rate'] = df_active['victory_count'] / df_active['total_games']

# 2. "Grinders" tanımı: 
# Kazanma oranı ortalamanın altında (%45 altı gibi) ama oyun süresi ortalamanın çok üstünde olanlar
win_rate_threshold = 0.45
duration_top_percentile = df_active['total_session_duration'].quantile(0.75)

grinders = df_active[
    (df_active['win_rate'] < win_rate_threshold) & 
    (df_active['total_session_duration'] > duration_top_percentile)
]


print(f"Analiz Edilen Aktif Oyuncu: {len(df_active)}")
print(f"Grinder (Yetenek düşük - Süre yüksek) Sayısı: {len(grinders)}")
print(f"Grinder Oranı: %{len(grinders)/len(df_active)*100:.2f}")

# 4. Görselleştirme
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_active.sample(min(3000, len(df_active))), 
                x='win_rate', y='total_session_duration', alpha=0.4)
plt.axvline(win_rate_threshold, color='red', linestyle='--', label='Kritik Win Rate (%45)')
plt.axhline(duration_top_percentile, color='purple', linestyle='--', label='Yüksek Süre Eşiği')
plt.title('Yetenek (Win Rate) vs. Zaman Yatırımı Analizi')
plt.xlabel('Kazanma Oranı (Win Rate)')
plt.ylabel('Toplam Oturum Süresi (Saniye)')
plt.legend()
plt.show()

In [ ]:
# Maç başına ortalama yenilgi (ölüm) sayısını hesaplayalım
# Not: total_session_count kolonunu maç sayısı olarak kullanıyoruz
df_active['deaths_per_match'] = df_active['defeat_count'] / df_active['total_session_count']

# "Feeder" tanımı: Maç başına yenilgi oranı en yüksek %10'luk dilim
feeder_threshold = df_active['deaths_per_match'].quantile(0.90)
feeders = df_active[df_active['deaths_per_match'] > feeder_threshold]

print(f"Feeder (Yüksek Ölüm Oranlı) Segmenti: {len(feeders)} kullanıcı")
print(f"Bu segmentin ortalama Win Rate'i: %{feeders['win_rate'].mean()*100:.2f}")

# Görselleştirme
plt.figure(figsize=(10, 6))
sns.histplot(df_active['deaths_per_match'], bins=50, kde=True, color='red')
plt.axvline(feeder_threshold, color='black', linestyle='--', label='Feeder Eşiği (Top %10)')
plt.title('Maç Başına Yenilgi Dağılımı ve Feeding Analizi')
plt.xlabel('Maç Başına Ortalama Yenilgi')
plt.ylabel('Oyuncu Sayısı')
plt.legend()
plt.show()

In [ ]:
# Yetenek (Win Rate) ve Zaman (Duration) Kıyaslaması
# 'Grinders' (Zamanla ilerleyenler) vs 'Elite' (Yetenekle ilerleyenler)
df_active['skill_group'] = 'Normal'
df_active.loc[(df_active['win_rate'] < 0.45) & (df_active['total_session_duration'] > duration_top_percentile), 'skill_group'] = 'Grinder'
df_active.loc[(df_active['win_rate'] > 0.60) & (df_active['total_session_duration'] > duration_top_percentile), 'skill_group'] = 'Elite'

# Bu grupların ortalama gelirlerini veya maç sayılarını kıyaslayalım
progression_summary = df_active.groupby('skill_group').agg({
    'total_session_duration': 'mean',
    'win_rate': 'mean',
    'iap_revenue': 'mean',
    'user_id': 'count'
})
print(progression_summary)

In [ ]:
# Harcama miktarına göre segmentasyon
def segment_users(revenue):
    if revenue > df_active['iap_revenue'].quantile(0.99): return 'Whale'
    elif revenue > 0: return 'Dolphin'
    else: return 'Minnow (Non-Payer)'

df_active['player_segment'] = df_active['iap_revenue'].apply(segment_users)

# Segmentlerin davranış analizi
segment_analysis = df_active.groupby('player_segment').agg({
    'total_session_duration': 'mean',
    'match_start_count': 'mean',
    'win_rate': 'mean',
    'user_id': 'count'
}).sort_values('total_session_duration', ascending=False)

print(segment_analysis)